# Computing Chi Square Values to determine the importance of each word in a corpus

### The corpus consists of multiple Amazon reviews belonging to different categories. https://jmcauley.ucsd.edu/data/amazon/ 

The aim is to first preprocess the dataset by splitting, tokenizing and filtering stop words (the stop word list is located under /lakobian/stopwords.txt but any stop word list works)

In [14]:
import pyspark
import re
import sys
from operator import add
import operator
import re
import pyspark.sql.functions as f
from collections import defaultdict
from pyspark.rdd import RDD
import heapq

sc = pyspark.SparkContext()


tf = sc.textFile("hdfs://c101.local:8020/user/lakobian/devset/reviews_devset.json")
sw = sc.textFile("hdfs://c101.local:8020/user/lakobian/stopwords.txt")
stopw = sw.flatMap(lambda x: x.split(" \n"))
stopwords = sc.broadcast(sw.collect())

ModuleNotFoundError: No module named 'pyspark'

In [17]:
!pip install pyspark

/bin/bash: pip: command not found


In [5]:
def preprocess(doc):
    category = doc[doc.find('\"category\":')+13:doc.find('"}')]
    text = doc[doc.find('\"reviewText\":')+14:doc.find('\"overall\"')]
    docs = re.split('[ \t\"\'`~#&*%$\\\\/0123456789.!?,;:()\\[\\]{}-]+',text)
    docs = list(filter(None, docs))
    docs = list(map(lambda word: word.lower(),docs))
    docs = list(set(filter(lambda word: word not in stopwords.value and (len(word) > 1), docs)))
    return (category,docs)

## the first mapper and reducer count the number of reviews in each category, and the count of a word in the corpus
def mapper_wordcount(row):
    review_text = row[1]
    category = row[0]
    
    document = []
    catisIncremented = 0
    if len(category)>1:
        for word in review_text:
            wordliste = []
            wordliste.append(word)
            wordliste.append(category)
            if not catisIncremented:
                wordliste.append(1)
                catisIncremented = 1
            else:
                wordliste.append(0)

            document.append(wordliste)
        
    return document


def reduce_wordcount(a,b):
    c = a.copy()
    if b.keys() <= c.keys():
        liste = c[list(b.keys())[0]]
        liste[1] += 2
        liste[0] += list(b.values())[0][0]
        c[list(b.keys())[0]] = liste
    else:
        c.update(b)
    return c


def split(doc):
    docs = re.split('[`~#&*%$\\](){}]+',doc)
    docs = list(filter(None, docs))
    overall_liste = []
    #del(docs[:1])
    word = docs[0].replace(", ", "")
    wordcount = 0
    for doc in docs[1:]:
        splits = re.split('[ \t\"\'`~#&*%$\\\\/.!?,;:()\\[\\]{}-]+',doc)
        splits = list(filter(None, splits))
        wordcount += int(splits[2])
    for doc in docs[1:]:
        liste = []
        splits = re.split('[ \t\"\'`~#&*%$\\\\/.!?,;:()\\[\\]{}-]+',doc)
        splits = list(filter(None, splits))
        #word = splits[0]
        liste.append(splits[0])
        liste.append(splits[1])
        liste.append(splits[2])
        liste.append(wordcount)
        liste.append(word)

        overall_liste.append(liste)
    return overall_liste

def mapper_catcount(row):
    
    category = row[0]
    info = row[1:]
    word = info[-1]
    liste = []
    liste += info[:-1]
    return (category,{word:liste})

def reduce_catcount(a,b):
    c = a.copy()
    if b.keys() <= c.keys():
        raise RuntimeError from exc
    else:
        c.update(b)
    return c


In [6]:
preprocessed_texts = tf.map(lambda doc: preprocess(doc))
N = preprocessed_texts.count()
mapped = preprocessed_texts.flatMap(lambda row: mapper_wordcount(row)).map(lambda row: (row[0],{row[1]:[row[2],1]}))
reduced = mapped.reduceByKey(lambda a, b: reduce_wordcount(a,b))

##Output directories need to be newly created! Change the name!
reduced.coalesce(1).saveAsTextFile("hdfs://c101.local:8020/user/lakobian/res1")

temp_input = sc.textFile("hdfs://c101.local:8020/user/lakobian/res1")
texts = temp_input.map(lambda doc: doc)

split_texts = texts.flatMap(lambda doc: split(doc)).map(lambda row: mapper_catcount(row))
reduced_texts = split_texts.reduceByKey(lambda a,b: reduce_catcount(a,b))
reduced_texts.coalesce(1).saveAsTextFile("hdfs://c101.local:8020/user/lakobian/res2")
temp_input2 = sc.textFile("hdfs://c101.local:8020/user/lakobian/res2")

NameError: name 'tf' is not defined

## Chi Square Computation 
as per http://www.ccs.neu.edu/home/vip/teach/DMcourse/3_dim_reduction/notes_slides/ChiSquare_FeatureSelection.pdf 

### For each category all words are sorted by their chi square value in descending order && top 150 words of each category are stored in a merged dictionary

In [ ]:
def split(doc):
    docs = re.split('[`~#&*%$\\](){}]+',doc)
    docs = list(filter(None, docs))
    overall_liste = []
    docs[0] = docs[0].replace(", ", "")
    P = 0
    chivalue = 0.0
    for doc in docs[1:]:
        split = re.split('[ \t\"\'`~#&*%$\\\\/.!?,;:()\\[\\]{}-]+',doc)
        split = list(filter(None, split))
        P += int(split[1])
    for doc in docs[1:]:
        liste = []
        splits = re.split('[ \t\"\'`~#&*%$\\\\/.!?,;:()\\[\\]{}-]+',doc)
        splits = list(filter(None, splits))
        #word = splits[0]
        A = float(splits[2])
        M = float(splits[3])
        B = (M - A)
        C = (P - A)
        D = (N - (A + B + C))
        try:
            chivalue = (float(N)*(abs((A*D)-(B*C))**2))//((A + C)*(B + D)*(A + B)*(C + D))
        except ZeroDivisionError:
            print(0)

        overall_liste.append((docs[0],[splits[0],chivalue]))
    return (overall_liste)


def orderByKey(self, num, sortValue = None):
 
        def init(a):
            return [a]
 
        def combine(agg, a):
            agg.append(a)
            return getTopN(agg)
 
        def merge(a, b):
            agg = a + b
            return getTopN(agg)
 
        def getTopN(agg):
            return heapq.nlargest(num, agg, sortValue)            
 
        return self.combineByKey(init, combine, merge)

    
RDD.orderByKey = orderByKey
top_entries = temp_input2.flatMap(lambda doc: split(doc)).orderByKey(150, sortValue=lambda x: x[1]).coalesce(1)
merged_dictionary = top_entries.flatMap(lambda a: a[1]).sortBy(lambda a: -a[1]).coalesce(1).map(lambda a: ("merged dictionary",a)).reduceByKey(lambda a,b:a+b)
both = top_entries.union(merged_dictionary)
both.saveAsTextFile("hdfs://c101.local:8020/user/lakobian/final_res")

sc.stop()

In [7]:
1+1

2

In [8]:
pwd

'/home/jpovh/Rscripts/big-data-training/code/Python'